## $\S$5. EM算法

### 5.1 高斯分布的背景

#### 一. 多元高斯分布

1. 多元高斯分布的概率密度函数 : ${ f }_{ X }({ x }_{ 1 }...{ x }_{ k })=\frac { 1 }{ \sqrt { { (2\pi ) }^{ k }|\Sigma | }  } exp\left[ -\frac { 1 }{ 2 } { (X-\mu ) }^{ T }{ \Sigma  }^{ -1 }(X-\mu ) \right] $  

#### 二. 应用场景  
1. 假设有一堆图片,主题分别是"海滩","天空","建筑".要把这些图片自动的分成3类(cluster后没有这些标签).一个思路是从图片的RGB值入手.  
2. 比如, 我们对每个图片的B(蓝色)占比画出直方图(histogram),发现不同label的图片在蓝色数值上展示出了不同的一元高斯分布
 <img src="../../img/gussian1.png" width="50%" height="50%">
3. 而当我们组合这三种图片的一元高斯分布直方图后, 形成一个起伏的联合直方图如下, 就是所谓的高斯混合.
  1. 3个不同随机变量组合为1个随机变量的本质, 是对他们进行凸组合.由于随机变量的概率之和为1,我们应该为这3个最初的随机变量分配权重  
  2. 用${ \Pi  }_{ k }$表示不同cluster的权重, ${ \Pi  }=\left[ { \pi  }_{ 1 },{ \pi  }_{ 2 },{ \pi  }_{ 3 } \right] $
<img src="../../img/gaussian2.png" width="90%" height="90%">

3. 以上所有图, 都只有Blue一个属性的分布, 这样做是为了简化表示这些图片的分类方式. 真正使用的应该是RGB三种颜色多元高斯分布建模.

#### 三. 如何应用高斯混合于文本聚类
1. 文本表示 : 使用标准的TF-IDF表示文本, 即每个文本向量属于${ R }^{ V },(V是单词表的长度)$
2. 协方差矩阵的学习  
 协方差矩阵在单词表长度V的条件下, 变为$\Sigma \in { R }^{ V }*{ R }^{ V }$, 由于$\Sigma$为对称矩阵(${ \Sigma  }_{ ij }={ \Sigma  }_{ ji }$) , 所以只需要学习上三角(或下三角)的参数即可, 此时参数个数达到$\frac { V(V+1) }{ 2 } $. 因此我们简单的初始化$\Sigma =\begin{bmatrix} { { \sigma  }_{ 1 } }^{ 2 } &  &  &  \\  & { { \sigma  }_{ 2 } }^{ 2 } &  &  \\  &  & ... &  \\  &  &  & { { \sigma  }_{ V } }^{ 2 } \end{bmatrix}$
3. 高斯混合比k-means灵活的地方  
 高斯混合可以学习不同维度的权重($R^V$), 即单词在单词表中的权重.   
 而kmeans只能把每个cluster边界认作相同大小的圆形, 即时在距离度量时分配了预先指定的权重, 最终得到的cluster边界也是相同比例的椭圆  
 <img src="../../img/gmm1.png" height="60%" width="60%">
 
### 5.2 EM推断 (expectation maximization)
#### 一. GMM计算方法
1. 高斯混合对观测样本, 给出所属每个cluster的概率, 这叫软分配. 表达式 : ${ r }_{ ik }=p\left( { { z }_{ i }=k }|{ { \left\{ { \pi  }_{ j },{ \mu  }_{ j },{ \Sigma  }_{ j } \right\}  }_{ j=1 }^{ k },{ x }^{ i } } \right) $  
  1. ${ z }_{ i }$ : 所求随即变量, 表示观测变量$i$ 分配到聚类k的概率
  2. ${ \left\{ { \pi  }_{ j },{ \mu  }_{ j },{ \Sigma  }_{ j } \right\}  }_{ j=1 }^{ k },{ x }^{ i }$ : 给定观测变量$x^i$和决定每个cluster边界形状的$({ \mu  }_{ j },{ \Sigma  }_{ j })$与每个cluster的先验概率${ \pi  }_{ j }$ 条件下  
2. ${ r }_{ ik }$的计算方法  
  1. ${ r }_{ ik }={ \pi  }_{ k }*N\left( { { x }^{ i } }|{ { \mu  }_{ k },{ \Sigma  }_{ k } } \right) $
  2. 上式计算每个${ x }^{ i }$归类为$K$的概率后, 为了使每个观测变量所有所属cluster的概率和为1, 还需对其进行正则化, 即${ r }_{ ik }=\frac { { \pi  }_{ k }*N\left( { { x }^{ i } }|{ { \mu  }_{ k },{ \Sigma  }_{ k } } \right)  }{ \sum _{ j=1 }^{ K }{ { \pi  }_{ j }*N\left( { { x }^{ i } }|{ { \mu  }_{ j },{ \Sigma  }_{ j } } \right)  }  } $
  
#### 二. 假设我们已经知道每个观测样本所属cluster的硬分配(带标签), 如何估计k个高斯分布的均值和方差参数
下面等式中, ${ N }_{ k }$表示应分配下,聚类k包含的观测变量个数. $N$: 所有的观测样本个数
1. 已知分布, 估计分布的参数值, 显然使用最大似然估计(MLE). MLE对参数的估计来自于对样本值的观察   
  1. 每个cluster内观测变量的均值 : ${ \hat { \mu  }  }_{ k }=\frac { 1 }{ { N }_{ k } } \sum _{ { x }^{ i }\in k }^{  }{ { x }^{ i } } $ :   
  2. 每个cluster内观测变量的方差 : ${ \hat { \Sigma  }  }_{ k }=\frac { 1 }{ { N }_{ k } } \sum _{ { x }^{ i }\in k }^{  }{ \left( { x }^{ i }-{ \widehat { \mu  }  }_{ k } \right) { \left( { x }^{ i }-{ \widehat { \mu  }  }_{ k } \right)  }^{ T } } $
2. ${ \pi  }_{ k }$的极大似然估计: $\hat { { \pi  }_{ k } } =\frac { { N }_{ k } }{ N } $

#### 三. 如果我们只知道每个观测变样本所属cluster的软分配, 如何估计每个cluster的参数
1. 软分配的意思是, 已知每个观测变量的${ r }_{ ik }$, 即每个${ x }^{ i }$所属cluster的概率. 这个数值也可以看做样本${ x }^{ i }$在每个cluster上的权重.因此, 在计算软分配下cluster参数的极大似然估计值时, 比硬分配多了权重  
2. 每个cluster在所有样本下的权值和 : ${ { N }_{ k } }^{ soft }=\sum _{ i=1 }^{ N }{ { r }_{ ik } } $   
 对比硬分配下的$N=\#\{{ x }^{ i }\in k\}$, 软分配下不在有确定的cluster所属, 而是权重
3. 软分配下参数估计带上权重 : $\begin{cases} { \hat { \mu  }  }_{ k }=\frac { 1 }{ { { N }_{ k } }^{ soft } } \sum _{ { x }^{ i }\in k }^{  }{ { r }_{ ik }*{ x }^{ i } }  \\ { \hat { \Sigma  }  }_{ k }=\frac { 1 }{ { { N }_{ k } }^{ soft } } \sum _{ { x }^{ i }\in k }^{  }{ { r }_{ ik }*\left( { x }^{ i }-{ \widehat { \mu  }  }_{ k } \right) { \left( { x }^{ i }-{ \widehat { \mu  }  }_{ k } \right)  }^{ T } }  \\ \hat { { \pi  }_{ k } } =\frac { { { N }_{ k } }^{ soft } }{ N }  \end{cases}$
4. 可以看出, 软分配下, cluster参数的极大似然估计还是直接计算观测值

### 5.3 EM算法
上面的讨论表明, 如果知道cluster的参数, 就可以计算软分配; 并且如果知道样本的软分配, 就能直接估计cluster的参数. 因此我们的算法要在这两部中不停迭代
#### 一. EM迭代算法
1. E-Step  
 给定cluster参数的情况下, 估计样本点在cluster下的软分配  
 ${ \hat { r }  }_{ ik }=\frac { { \hat { \pi  }  }_{ k }N({ x }_{ i }|{ \hat { \mu  }  }_{ k },{ \hat { \Sigma  }  }_{ k }) }{ \sum _{ j=1 }^{ K }{ { \hat { \pi  }  }_{ j }N({ x }_{ i }|{ \hat { \mu  }  }_{ j },{ \hat { \Sigma  }  }_{ j }) }  } $  
2. M-Step  
 给定cluster软分配的情况下, 对cluster参数进行极大似然估计  
 ${ \hat { \pi  }  }_{ k },{ \hat { \mu  }  }_{ k },{ \hat { \Sigma  }  }_{ k }|\{ { \hat { r }  }_{ ik },{ x }_{ i }\} $ 

#### 二. EM算法初始化
1. EM算法的收敛于局部解, 因此最终结果与初始值设置有很大关系. 一般使用K-Means先粗略聚类, 在用得到的centroid作为EM的初始化中心点进行精细估计. 
2. 此外, 初始化EM中心点的方法还有很多种, 这是一个值得探讨的问题. [参考: choose start value for EM](csda.pdf)

#### 三. EM与K-Means的关系
当协方差矩阵为对角矩阵时, cluster边界不再是椭圆而是正圆形. 且具有以下性质  
1. 每个cluster都是相等的直径. 此时极大似然估计就变成了对聚类中心的距离函数(带入多远高斯概率密度函数)
2. 当方差区域0时, ${ r }_{ ik }$只等于1或0 , 变成硬分配